# Udemy Grabber

In [58]:
import requests
from requests.utils import urlparse
from bs4 import BeautifulSoup, NavigableString
from bs4.element import Tag
import pandas as pd
from random import randint
from time import sleep
import re
from datetime import datetime, timedelta
from urllib.parse import parse_qs, unquote
import json

In [2]:
headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
headers['Accept'] = 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8'
headers['Accept-Encoding'] = 'gzip, deflate'
headers['Upgrade-Insecure-Requests'] = '1'
headers['Accept-Language'] = 'de;q=0.7'

sess = requests.Session()
sess.headers.update(headers)

## Gather Courses

In [10]:
def grab(url: str, timeout=10):
    return BeautifulSoup(sess.get(url, timeout=timeout).text, 'lxml')

### yofreesamples.com

In [4]:
soup = grab('https://yofreesamples.com/courses/free-discounted-udemy-courses-list/')
freesamples_courses = set(course['href'] for course in soup.find_all('a', class_='course_title', href=True))

In [5]:
print('Found {} courses.'.format(len(freesamples_courses)))

Found 50 courses.


### jucktion.com

In [6]:
jucktion = set()

for i in range(0, 7, 2):
    soup = grab('https://www.jucktion.com/forum/udemy-coupon/{}0'.format(i))
    for link in soup.find_all('td', class_='subject windowbg2'):
        l = link.find('a', href=True)['href']
        jucktion.add(l[:l.find('?PHP')])

jucktion_courses = set([grab(course).find('a', class_='bbc_link')['href'] for course in jucktion])

In [7]:
print('Found {} courses.'.format(len(jucktion_courses)))

Found 44 courses.


### bez-postovneho.cz

In [11]:
bez = set()
bez_courses = set()
for i in range(1, 6):
    soup = grab('https://bez-postovneho.cz/?page={}'.format(i))
    links = soup.find_all(href=re.compile("free-udemy-course/"))
    bez = bez.union([link['href'] for link in links])

for course in bez:
    soup = grab(course, timeout=15)
    url = soup.find('a', class_='btn btn-success', href=True)['href']
    if url.find('couponCode') > -1:
        bez_courses.add(url)

In [12]:
print('Found {} courses.'.format(len(bez_courses)))

Found 45 courses.


### reddit

In [13]:
reddit_courses = set()
regex = re.compile("http[s]*://www.udemy.com/course/[\w\d\-]{5,}/[\w?\d/]*couponCode=[\w]{3,}")

page = sess.get('https://www.reddit.com/r/Udemy/.json')
for article in page.json()['data']['children']:
    if datetime.utcfromtimestamp(article['data']['created_utc']) > (datetime.now() - timedelta(10)):
        text = article['data']['selftext']
        urls = re.findall(regex, text)
        if urls:
            reddit_courses = reddit_courses.union(urls)

In [14]:
print('Found {} courses.'.format(len(reddit_courses)))

Found 7 courses.


### dsmenders

In [15]:
dsmenders = set()
dsmenders_courses = set()
for i in range(1, 6):
    soup = grab('https://tech.dsmenders.com/category/free-online-courses/page/{}/'.format(i))
    links = soup.find_all('h2', 'entry-title')
    dsmenders = dsmenders.union([link.find('a', href=True)['href'] for link in links])

for course in dsmenders:
    soup = grab(course)
    url = soup.find('a', href=True, target='_blank', class_=False, style=True)
    if url and url['href'].find('couponCode') > -1:
        dsmenders_courses.add(url['href'])

In [16]:
print('Found {} courses.'.format(len(dsmenders_courses)))

Found 28 courses.


### Smartbro

In [17]:
smartbro_courses = set()
soup = grab('https://smartybro.com/category/udemy-coupon-100-off/')
links = soup.find_all('a', href=re.compile("https://smartybro.com/202"))
smartbro = set([link['href'][:link['href'].rfind('/')] for link in links])

for course in smartbro:
    soup = grab(course)
    tag = soup.find('a', href=re.compile('udemy.com/course/[\S]+/?couponCode='))
    if tag:
        l = tag['href']
        smartbro_courses.add(l[:l.find('&')])

In [18]:
print('Found {} courses.'.format(len(smartbro_courses)))

Found 5 courses.


### couponscorpion

In [19]:
scorpion = set()
scorpion_courses = set()
for i in range(1, 10):
    soup = grab('https://couponscorpion.com/page/{}/'.format(i))
    for article in soup.find_all('div', 'info_in_dealgrid'):
        if article.find('span', 'grid_onsale').text == '100% OFF':
            scorpion.add(article.find('a', href=True, class_=False)['href'])

regex = re.compile("[\w]{20,}%3D&s=[\w]+&n=[\w]+")
for course in scorpion:
    soup = grab(course)
    for script in soup.find_all('script', {'type':False, 'data-cfasync':False, 'src':False}):
        token = re.findall(regex, script.text)
        if token:
            url = sess.get('https://couponscorpion.com/scripts/udemy/out.php?go=' + token[0]).url
            coupon = parse_qs(urlparse(url).query)['couponCode']
            scorpion_courses.add(url[:url.find('?')] + '?couponCode=' + str(coupon[0]))

In [20]:
print('Found {} courses.'.format(len(scorpion_courses)))

Found 28 courses.


### discudemy

In [21]:
discudemy = set()
discudemy_courses = set()

for i in range(1, 6):
    soup = grab('https://www.discudemy.com/all/{}'.format(i))
    for div in soup.find_all('div', 'content'):
        if div.find('div', 'header'):
            if div.find('div', 'meta').text.find('->') > 0:
                discudemy.add(div.find('a', href=True)['href'])

regex = re.compile("/[\w\d\-]+/")
for course in discudemy:
    soup = grab(regex.sub('/go/', course))
    discudemy_courses.add(soup.find('a', href=True, class_=False, target='_blank')['href'])

In [22]:
print('Found {} courses.'.format(len(discudemy_courses)))

Found 30 courses.


### Facebook

In [23]:
groups = ['https://www.facebook.com/groups/FreeUdemyCoursesOnline/',
          'https://www.facebook.com/groups/freeudemycouponscourses/',
          'https://www.facebook.com/groups/1602890986642463/',
          'https://www.facebook.com/groups/FreeUdemyCoursesOnline/',
          'https://www.facebook.com/Udemy.Bargains/',
          'https://www.facebook.com/FreeOnlineCoursesCoupon/',
          'https://www.facebook.com/groups/DiscountedUdemyCoursesOnline',
          'https://web.facebook.com/groups/677040975746787',
          'https://web.facebook.com/groups/eLearningTrainingCourses',
          'https://web.facebook.com/groups/OnlineCoursesUdemy/',
          'https://web.facebook.com/groups/427365844137526',
          'https://web.facebook.com/groups/BestUdemyCourses',
          'https://web.facebook.com/groups/freecoursesudemy',
          'https://web.facebook.com/groups/1858168261178187',
          'https://web.facebook.com/groups/freeanddiscountedudemycoursecoupons',
         ]

In [24]:
facebook = set()
regex = re.compile('https%3A%2F%2Fwww.udemy.com%2Fcourse%2F[\S]+couponCode[=|%3D][\w-]{3,}')
for group in groups:
    page = sess.get(group).text
    for link in re.findall(regex, page):
        facebook.add(unquote(link))
    sleep(randint(1,2))

In [25]:
print('Found {} courses.'.format(len(facebook)))

Found 97 courses.


### Merge Courses

In [26]:
udemy = scorpion_courses | reddit_courses | bez_courses | freesamples_courses | discudemy_courses | facebook | smartbro_courses | dsmenders_courses | jucktion_courses

In [27]:
print('Found {} courses.'.format(len(udemy)))

Found 185 courses.


## Analyse
Find courses on Udemy, verify that they are free but not permanently, and crawl metadata. Use `sleep` to prevent Udemy from blocking us.

In [81]:
error_set = set()
def get_course_info(link):
    soup = grab(link).find('body')
    try:
        parsed = json.loads(soup.find('script', {'type':'application/ld+json'}).text)
    except AttributeError:
        print('Could not parse', link)
        return None

    meta = parsed[0]
    # currently free but not permanently

    try:
        price = soup.find('span', 'udlite-sr-only').next.next.next
    except:
        print('Course invalid:', link)
        return None

    if isinstance(price, NavigableString):
        price = price
    elif isinstance(price, Tag):
        price = price.text
    else:
        print(type(price))
        print(soup.find('span', 'udlite-sr-only').next.next)

    if price == 'Kostenlos':
        free = True
    elif price.find('€') > -1:
        free = False
    else:
        print('price not found')
        free = False

    if free and not meta['isAccessibleForFree']:
        rating, num_ratings, title, topics, language = parse_meta(meta, parsed)

        expiration = 99
        try:
            duration_text = soup.find('span', {'data-purpose': 'video-content-length'}).text.strip()
            duration = duration_text[:duration_text.rfind(' ')]
            expiration_text = soup.find('svg', {'aria-label':'alarm'}).next.next.next.next.text

            if not expiration_text.find('Tage') > -1:
                if expiration_text.find('Stunden') > -1:
                    expiration = int(expiration_text[:expiration_text.find('Stunden')])
                elif expiration_text.find('Minuten') > -1:
                    expiration = 0
        except AttributeError:
            # error when the duration or expiration element is missing
            # print('duration or expiration missing for', link)
            duration = 'fehlt'

        data = {'title': title,
                'link': link,
                'rating': rating,
                'num_ratings': num_ratings,
                'language': language,
                'duration': duration,
                'expiring': expiration
               }
        tops = dict(zip(['topic_' + str(t) for t in list(range(len(topics)))], topics))
        return {**data, **tops}

def parse_meta(meta, parsed):
    if 'aggregateRating' in meta.keys():
            rating = float(meta['aggregateRating']['ratingValue'])
            num_ratings = int(meta['aggregateRating']['ratingCount'])
    else:
        rating = 0
        num_ratings = 0

    title = meta['name']
    topics = [bc['name'] for bc in parsed[-1]['itemListElement']]
    language = meta['inLanguage']

    return rating, num_ratings, title, topics, language

In [82]:
day_before = pd.read_csv('./udemy-courses/2020-02-26 22-50.csv')
old_courses = pd.read_csv('./udemy-courses/2020-02-25 23-08.csv')

In [83]:
df = pd.DataFrame(columns=['title', 'link', 'rating', 'num_ratings', 'language',
                           'duration', 'topic_0', 'topic_1', 'topic_2', 'expiring'])

for link in set(day_before.link).union(udemy):
    result = get_course_info(link)
    df = df.append(result, ignore_index=True)
    sleep(randint(1,2))

Course invalid: https://www.udemy.com/course/network-attacks-and-security/?couponCode=1_LAKH_FREE
Could not parse https://www.udemy.com/course/microsoft-power-bi-latest-2020-beginner-to-expert-modules/?couponCode=N2M3B1
Course invalid: https://www.udemy.com/course/wi-fi-hacking-with-kali/?couponCode=1_LAKH_FREE_
Course invalid: https://www.udemy.com/course/xss-cross-site-scripting/?couponCode=1_LAKH_FREE


In [84]:
error_set

set()

In [86]:
#for link in list(error_set):
#    df = df.append(get_course_info(link), ignore_index=True)
#    sleep(randint(1,2))

## Find valuable Courses with high rating

In [87]:
valuable = df[(df.rating > 3) & (df.num_ratings > 8)].sort_values(by=['language', 'topic_0', 'topic_1'])

In [88]:
valuable

,title,link,rating,num_ratings,language,duration,topic_0,topic_1,topic_2,expiring
108,Wie Du Ein Erfolgreiches CPA Marketing Busines...,https://www.udemy.com/course/cpa-online-market...,3.8,50,de,"4,5 Stunden",Business,Selbstständigkeit,CPA-Marketing,19
47,Der komplette Listenaufbau-Kurs im eMail Marke...,https://www.udemy.com/course/listenaufbau-emai...,4.0,50,de,"8,5 Stunden",Marketing,Digitales Marketing,List-Building,19
18,Data Visualization in Excel: All Excel Charts ...,https://www.udemy.com/course/data-visualizatio...,4.3,30,en,"4,5 Stunden",Business,Daten & Analyse,Excel-Diagramme,17
96,Stock Options Day Trading Mindset for Success,https://www.udemy.com/course/stock-options-day...,3.7,59,en,"4,5 Stunden",Business,Finanzen,Aktienhandel,99
78,Transformational Leadership - Leadership Cours...,https://www.udemy.com/course/transformational-...,4.0,95,en,5 Stunden,Business,Management,Führung,19
...,...,...,...,...,...,...,...,...,...,...
83,10 Product Creation Hacks That Work In 2020,https://www.udemy.com/course/10-product-creati...,3.9,38,en,32 Min.,Wissenschaft & Lehre,Online-Unterricht,Online-Kurs-Erstellung,19
16,Multiple Choice Questions- Civil Engineering,https://www.udemy.com/course/basic-civil-engin...,4.1,9,en,fehlt,Wissenschaft & Lehre,Technische Wissenschaften,Bauwesen,99
98,Aprende a programar con Go (Golang),https://www.udemy.com/course/aprende-a-program...,4.3,223,es,"27,5 Stunden",Programmierung,Programmiersprachen,Programmierung (Grundlagen),99
94,Aprenda a fazer o melhor Podcast com Power Pod...,https://www.udemy.com/course/learn-podcasting-...,4.3,9,pt,3 Stunden,Marketing,Social-Media-Marketing,Podcasts,99


In [89]:
path = datetime.now().strftime('%Y-%m-%d %H-%M')
valuable.to_csv('./udemy-courses/' + path + '.csv')

In [90]:
yesterdays_courses = [course.title for course in day_before.itertuples() if not course.title in list(old_courses.title)]
new_courses = [course.title for course in valuable.itertuples() if not course.title in list(day_before.title)]

In [91]:
languages = valuable.language.unique()

In [92]:
print(languages)

['de' 'en' 'es' 'pt' 'tr']


## Edit Deal
* login to mydealz
* manually open the deal and delete the old list
* run script to create a new list

In [94]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.keys import Keys

In [95]:
driver = Firefox()

In [96]:
driver.get('https://www.mydealz.de/user/social/provider/google')

In [97]:
bold = driver.find_element_by_xpath("//button[@class='toolbar-btn toolbar-btn--bold space--mr-1']/span")
li = driver.find_element_by_xpath("//button[@class='toolbar-btn toolbar-btn--unorderedlist space--mr-1']/span")
link = driver.find_element_by_xpath("//button[@class='cept-linkBtn space--mr-1']/span")
editor = driver.find_element_by_xpath("//div[@class='redactor-box aGrid']/div")
editor.click()
editor.send_keys(Keys.CONTROL, Keys.END)

In [98]:
lang = {'de': 'Deutsch',
        'en': 'Englisch',
        'fr': 'Französisch',
        'es': 'Spanisch',
        'tr': 'Türkisch',
        'pt': 'Portugiesisch'}

In [99]:
def list_courses(courses):
    li.click()
    for course in courses.itertuples():
        if course.title in yesterdays_courses:
            editor.send_keys('*gestern ')
        elif course.title in new_courses:
            editor.send_keys('*neu ')
        if course.expiring < 99:
            editor.send_keys('🕖 ')
        if course.rating > 4.5:
            bold.click()
        link.click()
        driver.find_element_by_id('linkPopover-href').send_keys(course.link)
        driver.find_element_by_id('linkPopover-content').send_keys(course.title)
        driver.find_element_by_id('linkPopover-content').send_keys(Keys.RETURN)
        if course.rating > 4.5:
            bold.click()
        rounded = float(round(course.rating * 2, 0)/2)
        stars = '★' * int(rounded) + '✯' * bool(rounded % 1)
        stars += '☆' * (5 - len(stars))
        num_ratings = '({} Bewertungen)'.format(int(course.num_ratings))
        length = course.duration
        editor.send_keys(', ' + stars + ' ' + num_ratings + ', ' + length)
    
        editor.send_keys(Keys.RETURN)
        sleep(0.5)
    editor.send_keys(Keys.RETURN)

In [100]:
for language in languages:
    bold.click()
    editor.send_keys(lang[language])
    bold.click()
    editor.send_keys(Keys.ENTER)

    val_lang = valuable[valuable.language == language]
    main_topics = val_lang.topic_0.unique()
    for main_topic in main_topics:
        editor.send_keys(main_topic)
        editor.send_keys(Keys.RETURN)
        list_courses(val_lang[valuable.topic_0 == main_topic])
    editor.send_keys(Keys.RETURN)

D:\Programme\Anaconda\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [101]:
for course in valuable.itertuples():
    print(course.link)

https://www.udemy.com/course/cpa-online-marketing-kurs/?couponCode=E3D9D93D77D6A0787742
https://www.udemy.com/course/listenaufbau-email-marketing-und-newsletter-kurs/?couponCode=D56812240E38A51DB465
https://www.udemy.com/course/data-visualization-in-excel-for-business-professionals/?couponCode=FEB2020FREE
https://www.udemy.com/course/stock-options-day-trading-success-mindset/?couponCode=F183514155225E1E8DFD
https://www.udemy.com/course/transformational-leadership-training-course/?couponCode=E14296C8EB327EBB2FBC
https://www.udemy.com/course/how-to-validate-your-idea-and-succeed-in-business/?couponCode=8D89C65A9052A040250C
https://www.udemy.com/course/ultimate-time-management-best-time-management-course-class-training/?couponCode=432989C22969F29DBB65
https://www.udemy.com/course/outstanding-customer-service/?couponCode=E262DB6BA4BDF7E62578
https://www.udemy.com/course/business-education-guide-to-blockchain-and-cryptocurrencies/?couponCode=6073A48FABB1DC5DA4D0
https://www.udemy.com/course

## Subscribe to Courses
Login manually

In [105]:
driver.quit()
driver = Firefox()

In [106]:
driver.get('https://www.udemy.com')
driver.find_element_by_xpath("//button[@class='btn btn-quaternary']").click()

In [ ]:
new_links = [course.link for course in valuable.itertuples() if not course.title in list(day_before.title)]
for link in new_links:
    driver.get(link)
    try:
        sleep(randint(4, 10))
        btn = driver.find_element_by_xpath("//button[@data-purpose='buy-this-course-button']")
        if btn.text == 'Jetzt einschreiben':
            btn.click()
            sleep(6)
        else:
            print(link, 'already subscribed.')
    except:
        print(link, '# already subscribed.')

In [657]:
driver.quit()